## UPF Expeiments
    Feb2024

In [1]:

import os
import pandas as pd
import numpy as np
from sklearn.metrics import (accuracy_score, f1_score, precision_score, 
                             recall_score, roc_auc_score, mean_squared_error, r2_score)
from sklearn.model_selection import train_test_split
import xgboost as xgb
import math
import shap
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt


from config import region_mapping, set1, to_model_a, to_model_c, to_model_ac, to_model, set10


path = '/Users/jerzybala/Desktop/Simulation Experiments Aug-Sep 2023/PROCESSED_FOOD_Dec2023.csv'

path_temp = '/Volumes/2TB Ext/BrainBaseMHQ/mhm_data_2024-02-03_17-00-42_from jan2023.csv'

path_temp_processed = '/Volumes/2TB Ext/BrainBaseMHQ/GMP_data_2023_to_jan2024.csv'

df = pd.read_csv(path, low_memory=False)





df.loc[:, 'Processed food in diet'] = df['Processed food in diet'].replace({
            'Rarely/never': 'Rarely/Never',
            'A few times in a day': 'Several times a day',
            'Several days a week': 'A few times a week',
            'Many times in a day': 'Several times a day',
            'At least once a day': 'Several times a day'
        })


print(f"Data loaded. Shape: {df.shape}")



Data loaded. Shape: (401682, 239)


In [2]:

df_features = df[set10].copy()

## Function Definitions

In [4]:
def segment_and_prepare_df(df, countries=None, age_groups=None, drop_list=None):
    """
    Segments the DataFrame based on specific countries and age groups, and then drops specified columns.
    
    Parameters:
    - df: DataFrame to be processed.
    - countries: List of countries for segmentation. If None, no country-based segmentation is applied.
    - age_groups: List of age groups for segmentation. If None, no age-based segmentation is applied.
    - drop_list: List of columns to be dropped from the DataFrame.
    
    Returns:
    - A DataFrame that has been segmented and had specified columns dropped.
    """
    # Segment DataFrame based on provided countries and age groups
    segmented_df = df.copy()
    if countries is not None:
        segmented_df = segmented_df[segmented_df['Country'].isin(countries)]
    if age_groups is not None:
        segmented_df = segmented_df[segmented_df['Age'].isin(age_groups)]
    
    # Drop specified columns if drop_list is provided
    if drop_list is not None:
        segmented_df = segmented_df.drop(columns=drop_list)
    
    return segmented_df
#================================================================================================


def encode_features(df, categorical_features):
    """Encodes categorical features using one-hot encoding."""
    encoded_features = pd.get_dummies(df[categorical_features])
    return pd.concat([df.drop(columns=categorical_features), encoded_features], axis=1)
#================================================================================================


def prepare_dataset(df, target_classification, target_regression):
    """Prepares the dataset by dropping specified columns, encoding categorical features, and splitting into features and targets."""
    #df_filtered = drop_columns(df, drop_list)
    categorical_features = df.select_dtypes(include=['object']).columns
    df_filtered = encode_features(df, categorical_features)
    
    X = df_filtered.drop(columns=[target_classification, target_regression], axis=1)
    y_classification = df_filtered[target_classification]
    y_regression = df_filtered[target_regression]
    
    return X, y_classification, y_regression

#================================================================================================

def split_dataset(X, y_classification, y_regression, test_size=0.3, random_state=42):
    """Splits the dataset into training and testing sets for both classification and regression tasks."""
    X_train, X_test, y_train_class, y_test_class = train_test_split(X, y_classification, test_size=test_size, random_state=random_state)
    X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y_regression, test_size=test_size, random_state=random_state)
    
    return X_train, X_test, y_train_class, y_test_class, X_train_reg, X_test_reg, y_train_reg, y_test_reg

#================================================================================================





In [5]:
drop_list = [

    'Country',

    'Household Income',

    'ARCHIVED: Smartphone ownership',
    'ARCHIVED: Age of smartphone access',
    'Smartphone allowed in school',
    'Smartphone use in lessons',

    'Smartphone ownership',
    'Friends/classmates smarphone ownership',
    'Age of smartphone usage during school hours',
    'Smartphone usage during class hours',
    'Smartphone usage during break'

]

anglosphere = ['United States', 'United Kingdom', 'Canada', 'Australia', 'New Zealand', 'Ireland']
kraj = ['United States']
kraj2 =['India']
wiek = ['18-24', '21-24', '18', '19', '20']
wiek2 = ['55-64','45-54']


# Segment and prepare the DataFrame
prepared_df = segment_and_prepare_df(df_features, countries=anglosphere, age_groups=None, drop_list=drop_list)


prepared_df.shape

(39375, 46)

In [ ]:
#prepared_df.columns.to_list()

### New Features

In [6]:
Legal_Substance_Use = [
    'Tobacco products',
    'Alcoholic beverages',
    'Cannabis',
    'Vaping products',
    'Sedatives or Sleeping Pills',
    'Amphetamine type stimulants (e.g. speed| diet pills| ecstasy| etc.)',
    'Opioids',
    'Melatonina',
    'Cigarrillo'
]

Interpersonal_Trauma = [
    'Prolonged emotional or psychological abuse or neglect from parent/caregiver',
    'Physical violence in the home between family members',
    'Prolonged or sustained bullying in person from peers',
    'Prolonged physical abuse| or severe physical assault CT',
    'Threatening| coercive or controlling behavior by another person CT',
    'Threatening| coercive or controlling behavior by another person',
    'Prolonged sexual abuse| or severe sexual assault.',
    'Cyberbullying or online abuse',
    'Forced family control over major life decisions (e.g. marriage)',
    'Forced family control over major life decisions CT'
]

Life_Adversities = [
    'Sudden or premature death of a loved one',
    'Parental Divorce or family breakup',
    'Divorce/separation  or family breakup',
    'Extreme poverty leading to homelessness and/or hunger.',
    'Sudden or premature death of a parent or sibling',
    'Loss of your job or livelihood leading to an inability to make ends meet.',
    'Lived with a parent/caregiver who was an alcoholic or who regularly used street drugs',
    'Life threatening or debilitating injury or illness.',
    'Suffered a loss in a major fire| flood| earthquake| or natural disaster',
    'Displacement from your home due to political| environmental or economic reasons CT',
    'Suffered a loss in a major fire| flood| earthquake| or natural disaster CT',
    'Life threatening or debilitating injury or illness CT',
    'Displacement from your home due to political| environmental or economic reasons',
    'Involvement or close witness to a war',
    'Caring for a child or partner with a major chronic disability or illness'
]


prepared_df['Legal_Substance_Use'] = prepared_df[Legal_Substance_Use].sum(axis=1)
prepared_df['Interpersonal_Trauma'] = prepared_df[Interpersonal_Trauma].sum(axis=1)
prepared_df['Life_Adversities'] = prepared_df[Life_Adversities].sum(axis=1)


prepared_df.to_csv('/Users/jerzybala/Desktop/prepared_df.csv', index=False)



## Feature Exploreation

In [ ]:
prepared_df.columns.to_list()

In [7]:
# Include or not to include UPF

prepared_df=prepared_df.drop('Processed food in diet', axis=1)


In [ ]:
import numpy as np
from scipy.stats import entropy

def calculate_infogain(df, target):
    # Calculate the entropy of the target variable
    target_entropy = entropy(df[target].value_counts(normalize=True), base=2)
    
    # Calculate the entropy of each feature
    feature_entropies = []
    for column in df.columns:
        if column != target:
            feature_entropy = 0
            for value in df[column].unique():
                subset = df[df[column] == value]
                subset_entropy = entropy(subset[target].value_counts(normalize=True), base=2)
                feature_entropy += (len(subset) / len(df)) * subset_entropy
            feature_entropies.append(feature_entropy)
    
    # Calculate the information gain for each feature
    infogains = target_entropy - np.array(feature_entropies)
    
    # Create a dictionary mapping each feature to its information gain
    infogain_dict = dict(zip(df.columns[df.columns != target], infogains))
    
    return infogain_dict


In [ ]:
prepared_df_c =prepared_df.drop(columns='Overall MHQ', axis=1)


info_gained_features = calculate_infogain(prepared_df_c, 'MHQ_Sign')


In [ ]:
from tabulate import tabulate

def print_sorted_feature_scores(feature_scores, score_name="Score", table_format="grid"):
    """
    Sorts a dictionary of features and their scores in descending order and prints it in tabulated format.

    Parameters:
    - feature_scores: Dictionary with features as keys and scores as values.
    - score_name: A string representing the name of the score for the header. Defaults to "Score".
    - table_format: A string representing the table format to be used by tabulate. Defaults to "grid".
    """
    # Convert the dictionary to a list of tuples and sort it in descending order by score
    feature_scores_sorted = sorted(feature_scores.items(), key=lambda x: x[1], reverse=True)

    # Print the sorted list in tabulate format with customizable headers
    print(tabulate(feature_scores_sorted, headers=["Feature", score_name], tablefmt=table_format))

# Example usage for information gain:
print_sorted_feature_scores(info_gained_features, score_name="Information Gain")




In [ ]:
## Write to Excel

# Convert the dictionary to a DataFrame
features_ranks = pd.DataFrame(list(info_gained_features.items()), columns=['Feature', 'Information Gain'])

# Sort the DataFrame by 'Information Gain' in descending order
features_ranks_sorted = features_ranks.sort_values(by='Information Gain', ascending=False)

# Specify the file path (adjust the path as necessary for your system)
file_path = '/Users/jerzybala/Desktop/sapien_temp.xlsx'

# Write the sorted DataFrame to an Excel file
features_ranks_sorted.to_excel(file_path, index=False, engine='openpyxl')

print(f"Feature scores have been written to {file_path}")




In [ ]:
# File deletion

#  CAREFUL! This will delete the file at the specified path
# ******************************************************

# After ensuring the file is no longer needed, delete it
os.remove(file_path)
print(f"The file {file_path} has been deleted.")


In [ ]:

list1 = ['Interpersonal_Trauma', 'Age', 'Legal_Substance_Use', 'Life_Adversities']


for l in list1:
    a = prepared_df[l].unique().tolist()
    print(f"{l}: {a}, number of unique={len(a)}")   




In [ ]:
prepared_df.to_csv('/Users/jerzybala/Desktop/prepared_df.csv', index=False)

In [8]:

X, y_classification, y_regression = prepare_dataset(prepared_df, 'MHQ_Sign', 'Overall MHQ')

X_train, X_test, y_train, y_test, X_train_reg, X_test_reg, y_train_reg, y_test_reg = split_dataset(
    X, 
    y_classification, 
    y_regression, 
    test_size=0.3, 
    random_state=42
)


In [9]:


par1 = {
    'n_estimators': 200, 
    'learning_rate': 0.01,
    'max_depth': 10, 
    'min_child_weight': 1, 
    'gamma': 0.01, 
    'subsample': 0.8,
    'colsample_bytree': 0.8
}

# Initialize XGBoost classifier and train

model_C = xgb.XGBClassifier(**par1)
model_C.fit(X_train, y_train)

# Predict and evaluate
predictions = model_C.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
auc = roc_auc_score(y_test, model_C.predict_proba(X_test)[:, 1])

print("xgb.XGBClassifier:")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"AUC: {auc}")
print("\n")


#==================================================================================================


# Initialize XGBoost regressor and train
#model = xgb.XGBRegressor()
# X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y_regression, test_size=0.3, random_state=42)


#model_R = xgb.XGBRegressor()

model_R = xgb.XGBRegressor(
n_estimators=500,
learning_rate=0.1,
max_depth=6,
min_child_weight=1,
gamma=0.2,
subsample=0.4,
colsample_bytree=0.8,
#reg_alpha=10,
reg_lambda=0.1
)

model_R.fit(X_train_reg, y_train_reg)

# Separate predictions based on the sign of y_test
predictions = model_R.predict(X_test_reg)

mse = mean_squared_error(y_test_reg, predictions)
rmse = math.sqrt(mse)
r2 = r2_score(y_test_reg, predictions)
mae = mean_absolute_error(y_test_reg, predictions)

print("xgb.XGBRegressor:")
print('mae:', mae)
print('rmse:',rmse)
print('r2:', r2)
print("\n")



xgb.XGBClassifier:
Accuracy: 0.8214678743756878
F1 Score: 0.8932638291411509
Precision: 0.8363343442001516
Recall: 0.9585098294775715
AUC: 0.8322790244100866


xgb.XGBRegressor:
mae: 44.94170743862004
rmse: 56.44146975569511
r2: 0.40346480687535413




# TEMP

In [ ]:
def segment_and_prepare_df(df, countries=None, age_groups=None, drop_list=None):
    """
    Segments the DataFrame based on specific countries and age groups, and then drops specified columns.
    
    Parameters:
    - df: DataFrame to be processed.
    - countries: List of countries for segmentation. If None, no country-based segmentation is applied.
    - age_groups: List of age groups for segmentation. If None, no age-based segmentation is applied.
    - drop_list: List of columns to be dropped from the DataFrame.
    
    Returns:
    - A DataFrame that has been segmented and had specified columns dropped.
    """
    # Segment DataFrame based on provided countries and age groups
    segmented_df = df.copy()
    if countries is not None:
        segmented_df = segmented_df[segmented_df['Country'].isin(countries)]
    if age_groups is not None:
        segmented_df = segmented_df[segmented_df['Age'].isin(age_groups)]
    
    # Drop specified columns if drop_list is provided
    if drop_list is not None:
        segmented_df = segmented_df.drop(columns=drop_list)
    
    return segmented_df




In [ ]:
# Assuming df is your initial dataframe
countries = ['United States', 'United Kingdom', 'Canada', 'Australia', 'New Zealand', 'Ireland', 'South Africa']
age_groups = ['18-24', '21-24', '18', '19', '20']
drop_list = [
    'Country',
    'Smartphone ownership',
    'Friends/classmates smartphone ownership',
    'Age of smartphone usage during school hours',
    'Smartphone usage during class hours',
    'Smartphone usage during break'
]

# Segment and prepare the DataFrame
prepared_df = segment_and_prepare_df(df, countries=countries, age_groups=age_groups, drop_list=drop_list)

# Continue with further data preparation steps as needed

In [ ]:
def prepare_dataset(df, drop_list, target_classification, target_regression):
    """Prepares the dataset by dropping specified columns, encoding categorical features, and splitting into features and targets."""
    df_filtered = drop_columns(df, drop_list)
    categorical_features = df_filtered.select_dtypes(include=['object']).columns
    df_filtered = encode_features(df_filtered, categorical_features)
    
    X = df_filtered.drop(columns=[target_classification, target_regression], axis=1)
    y_classification = df_filtered[target_classification]
    y_regression = df_filtered[target_regression]
    
    return X, y_classification, y_regression

def split_dataset(X, y_classification, y_regression, test_size=0.3, random_state=42):
    """Splits the dataset into training and testing sets for both classification and regression tasks."""
    X_train, X_test, y_train_class, y_test_class = train_test_split(X, y_classification, test_size=test_size, random_state=random_state)
    X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y_regression, test_size=test_size, random_state=random_state)
    
    return X_train, X_test, y_train_class, y_test_class, X_train_reg, X_test_reg, y_train_reg, y_test_reg





In [ ]:
# Example of how to use these functions
drop_list = [
    'Country',
    'Smartphone ownership',
    'Friends/classmates smartphone ownership',
    'Age of smartphone usage during school hours',
    'Smartphone usage during class hours',
    'Smartphone usage during break'
]

# Assuming df is your initial dataframe
X, y_classification, y_regression = prepare_dataset(df, drop_list, 'MHQ_Sign', 'Overall MHQ')
X_train, X_test, y_train_class, y_test_class, X_train_reg, X_test_reg, y_train_reg, y_test_reg = split_dataset(X, y_classification, y_regression)

In [ ]:



df_processed = df[set10].copy()

df_processed.loc[:, 'Processed food in diet'] = df_processed['Processed food in diet'].replace({
            'Rarely/never': 'Rarely/Never',
            'A few times in a day': 'Several times a day',
            'Several days a week': 'A few times a week',
            'Many times in a day': 'Several times a day',
            'At least once a day': 'Several times a day',
            'None of the above': 'None of the above SU'
        })




# Filter the encoded dataframe
anglosphere = ['United States', 'United Kingdom', 'Canada', 'Australia', 'New Zealand', 'Ireland']


kraj = ['United States']
kraj2 =['India']
        

wiek = ['18-24', '21-24', '18', '19', '20']
wiek2 = ['55-64','45-54']


#df_filtered = df_processed[(df_processed['Country'].isin(kraj)) & (df_processed['Age'].isin(wiek))]


#(2)
#df_filtered = df_processed[(df_processed['Country'].isin(anglosphere))]

#(1)
#df_filtered = df_processed.copy()


#(3) 
#df_filtered = df_processed[(df_processed['Country'].isin(kraj))]


#4 & 5#
#df_filtered = df_processed[(df_processed['Country'].isin(kraj)) & (df_processed['Age'].isin(wiek))]



#df_filtered = df_processed[(df_processed['Age'].isin(wiek2))]



df_filtered.shape